In [7]:
!pip install langchain jq langchain-community langchain-openai

  Using cached tiktoken-0.8.0-cp312-cp312-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.8.2-cp312-cp312-macosx_10_12_x86_64.whl.metadata (5.2 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_10_13_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.8.0-cp312-cp312-macosx_10_13_x86_64.whl (1.0 MB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached jiter-0.8.2-cp312-cp312-macosx_10_12_x86_64.whl (302 kB)
Using cached regex-2024.11.6-cp312-cp312-macosx_10_13_x86_64.whl (288 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [1]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


In [76]:
from langchain_community.document_loaders import JSONLoader
import json
from pathlib import Path
import jq

file_path='./opendatahub.json'

with open(file_path) as f:
    data = json.load(f)
    api_url = jq.compile('.servers[0].url').input(data).first()
    print(api_url)

# data = json.loads(Path(file_path).read_text())
loader = JSONLoader(
         file_path=file_path,
         jq_schema='.paths | to_entries[] | .key as $path | .value | to_entries[] | { path:$path, method:.key, tag:.value.tags[0], summary:.value.summary, parameters: [ {name: .value.parameters[]?.name } ] }',
         text_content=False)

https://tourism.api.opendatahub.com


In [53]:
docs = loader.load()

In [54]:
len(docs)

152

In [71]:
docs[1]

Document(metadata={'source': '/Users/user/Projects/genai4se-course/opendatahub.json', 'seq_num': 2}, page_content='{"path": "/v1/Accommodation", "method": "post", "tag": "Accommodation", "summary": "POST Insert new Accommodation", "parameters": []}')

In [56]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [57]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [58]:
ids = vector_store.add_documents(documents=docs)

In [59]:
ids

['75f9b7a2-db2c-405d-8709-bfee713ab79f',
 '44151b7a-87f3-4e39-a0db-e9d7c6132687',
 'c6dd2d4c-accc-4904-a2c0-2102e273113f',
 'bca9b476-9927-42d7-85c1-536603b7d455',
 '904ac7b6-bff6-4035-9cba-1553c2abfbd8',
 '3d214f0d-cc9f-450b-a7b0-a60a38954a73',
 '4c9c0f18-46bf-4b99-9c9b-9c1b489a75c6',
 '3e2ca526-32ba-4c58-9b63-20dcf548846a',
 '5151fe34-3c98-4425-98b5-e5cb512f66a9',
 '8bea5f91-e0a5-4751-8ae6-94fff988014f',
 '5d184343-7c85-4493-96ff-5a06f0e9a7ec',
 '7bf2c17c-fb01-4987-8b97-4e5b035672ef',
 'ae1920a4-ad00-4cef-8bc5-039e9ac9daf1',
 'c6181b6d-38c9-41f3-a119-298e24e71a39',
 '44b3cf95-bb57-42b9-83ca-d20c11c7cd9d',
 '39e8a4eb-883f-4dbe-af5e-678c9418a54c',
 'ad1e0e1a-99d2-4bea-abf0-64629c20da48',
 '0e19518b-85ab-49a1-8c4f-18b4fbae95cc',
 '98c1af33-3754-4285-8c4b-b2b3cd530860',
 '68bc1482-c3dd-4d6b-b8cd-4106f0eff4e2',
 '2434ca82-bba6-4dda-afc0-0b51d692f04e',
 '2f42fe11-b275-4466-a5c4-dc585c46ee54',
 '4f28ae1d-f838-4a56-9f4a-10c0f7bfa9ff',
 'ac03509d-ee26-43de-9e21-c79b92a61305',
 '8d5a198b-7926-

In [60]:
task = "Write a piece of code to list the events in the last year."

In [61]:
retrieved_docs = vector_store.similarity_search(task)
docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
docs_content

'{"path": "/v1/EventShort", "method": "get", "tag": "EventShort", "summary": "GET EventShort List", "parameters": [{"name": "pagenumber"}, {"name": "pagesize"}, {"name": "startdate"}, {"name": "enddate"}, {"name": "datetimeformat"}, {"name": "source"}, {"name": "eventlocation"}, {"name": "onlyactive"}, {"name": "websiteactive"}, {"name": "communityactive"}, {"name": "active"}, {"name": "eventids"}, {"name": "webaddress"}, {"name": "sortorder"}, {"name": "seed"}, {"name": "language"}, {"name": "langfilter"}, {"name": "optimizedates"}, {"name": "latitude"}, {"name": "longitude"}, {"name": "radius"}, {"name": "polygon"}, {"name": "fields"}, {"name": "lastchange"}, {"name": "publishedon"}, {"name": "searchfilter"}, {"name": "rawfilter"}, {"name": "rawsort"}, {"name": "removenullvalues"}]}\n\n{"path": "/v1/Event", "method": "get", "tag": "Event", "summary": "GET Event List", "parameters": [{"name": "language"}, {"name": "pagenumber"}, {"name": "pagesize"}, {"name": "idlist"}, {"name": "locf

In [88]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o-mini")

prompt_template = ChatPromptTemplate([
    ("system", "You are a developer using an API to implement a solution in Python."),
    ("user", "Below, there is the information about the API you need to use {apiInfo}. Your task is: {task}. Just return the code without anything else. The API URL is {apiUrl}")
]) 

prompt = prompt_template.invoke({"apiInfo": docs_content, "task": task, "apiUrl": api_url})
answer = llm.invoke(prompt)
result = answer.content

answer.pretty_print()

if "```python" in result:
        result = result[10:-3]

output_path = "output/main.py"
with open(output_path, "w") as f:
    f.write(result)

================================== Ai Message ==================================

```python
import requests
from datetime import datetime, timedelta

# Calculate the date range for the last year
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

# Format dates as needed (example: YYYY-MM-DD)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# Set up the parameters for the API request
params = {
    "startdate": start_date_str,
    "enddate": end_date_str,
    "onlyactive": True,
    "pagesize": 100,  # Example page size
    "pagenumber": 1    # Start with the first page
}

# Make the API call
response = requests.get("https://tourism.api.opendatahub.com/v1/EventShort", params=params)

# Check the response
if response.status_code == 200:
    events = response.json()
    print(events)  # List of events
else:
    print(f"Error: {response.status_code} - {response.text}")
```


In [93]:
!python3.10 output/main.py

{'TotalResults': 1002, 'TotalPages': 11, 'CurrentPage': 1, 'PreviousPage': None, 'NextPage': 'https://tourism.api.opendatahub.com/v1/EventShort?startdate=2024-01-29&enddate=2025-01-28&onlyactive=True&pagesize=100&pagenumber=2', 'Seed': None, 'Items': [{'Id': '6389d893-6c6a-437b-a370-1e77a4e67b75', 'Self': 'https://tourism.api.opendatahub.com/v1/EventShort/6389d893-6c6a-437b-a370-1e77a4e67b75', 'Tags': [{'Id': 'digital', 'Name': 'Digital', 'Self': 'https://tourism.api.opendatahub.com/v1/Tag/digital', 'Type': 'technologyfields', 'Source': 'idm'}, {'Id': 'techtransfer', 'Name': 'Tech Transfer', 'Self': 'https://tourism.api.opendatahub.com/v1/Tag/techtransfer', 'Type': 'customtagging', 'Source': 'idm'}, {'Id': 'public', 'Name': 'Public', 'Self': 'https://tourism.api.opendatahub.com/v1/Tag/public', 'Type': 'customtagging', 'Source': 'idm'}], '_Meta': {'Id': '6389d893-6c6a-437b-a370-1e77a4e67b75', 'Type': 'eventshort', 'Source': 'noi', 'Reduced': False, 'LastUpdate': '2024-02-22T12:28:32.592